# Phase 4 capstone project by Mayur Yeole
1) Write the complete Machine learning code to make predictions of votes and gross collection. Use appropriate models on their label basis. Remember you need to make 2 different predictions: vote and gross collection.

2) Apply all the best techniques of scaling, and hyperparameter tuning, and avoid underfitting or overfitting (bias/variance)

3) At the end save the best model and convey on which basis you have chosen that model.  

In [94]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

In [95]:
# Save the data to a new CSV file without commas in 'Gross Collection'
df.to_csv('merged_data_without_commas.csv', index=False)

In [121]:
# Print the dataframe to check the updated 'Gross Collection'
print(df)

    sno                                         Movie Name  Duration  Year  \
0     1                                    The Dark Knight       152  2008   
1     2      The Lord of the Rings: The Return of the King       201  2003   
2     3                Spider-Man: Across the Spider-Verse       140  2023   
3     4                                          Inception       148  2010   
4     5  The Lord of the Rings: The Fellowship of the Ring       178  2001   
5     6              The Lord of the Rings: The Two Towers       179  2002   
6     7                                         The Matrix       136  1999   
7     8     Star Wars: Episode V - The Empire Strikes Back       124  1980   
8     9                                    Soorarai Pottru       153  2020   
9    10                         Terminator 2: Judgment Day       137  1991   
10   11                                          Star Wars       121  1977   
11   12                               Shichinin no samurai      

In [122]:
import pandas as pd

# Load data from CSV file
data = pd.read_csv('merged_data_without_commas.csv')

# Split data into features (X) and labels (y) for votes and gross collection
X_votes = data[['Duration', 'Year', 'Ratings', 'Metascore']]
y_votes = data['Votes']

X_gross = data[['Duration', 'Year', 'Ratings', 'Metascore', 'Popularity']]
y_gross = data['Gross Collection']


In [123]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Handle missing values
X_votes.fillna(0, inplace=True)  # Replace NaN values with 0 for votes
X_gross.fillna(0, inplace=True)  # Replace NaN values with 0 for gross collection

# Scale the features
scaler_votes = StandardScaler()
X_votes_scaled = scaler_votes.fit_transform(X_votes)

scaler_gross = StandardScaler()
X_gross_scaled = scaler_gross.fit_transform(X_gross)

# Split data into training and testing sets
X_votes_train, X_votes_test, y_votes_train, y_votes_test = train_test_split(X_votes_scaled, y_votes, test_size=0.2, random_state=42)
X_gross_train, X_gross_test, y_gross_train, y_gross_test = train_test_split(X_gross_scaled, y_gross, test_size=0.2, random_state=42)

C:\Users\write\AppData\Local\Temp\ipykernel_8644\1676021454.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_votes.fillna(0, inplace=True)  # Replace NaN values with 0 for votes
C:\Users\write\AppData\Local\Temp\ipykernel_8644\1676021454.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_gross.fillna(0, inplace=True)  # Replace NaN values with 0 for gross collection


In [126]:
pip install xgboost


     ---------------------------------------- 70.9/70.9 MB 4.7 MB/s eta 0:00:00


In [127]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

# Define the XGBoost regressor
xgb_reg_votes = XGBRegressor()
xgb_reg_gross = XGBRegressor()

# Define the hyperparameter grid for tuning
param_grid = {
    'learning_rate': [0.1, 0.01],
    'max_depth': [3, 5, 7],
    'n_estimators': [100, 200, 300]
}

# Perform hyperparameter tuning using GridSearchCV for votes
grid_search_votes = GridSearchCV(estimator=xgb_reg_votes, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search_votes.fit(X_votes_train, y_votes_train)

# Perform hyperparameter tuning using GridSearchCV for gross collection
grid_search_gross = GridSearchCV(estimator=xgb_reg_gross, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search_gross.fit(X_gross_train, y_gross_train)

# Get the best parameters
best_params_votes = grid_search_votes.best_params_
best_params_gross = grid_search_gross.best_params_

# Update the models with the best parameters
xgb_reg_votes = XGBRegressor(**best_params_votes)
xgb_reg_gross = XGBRegressor(**best_params_gross)

# Fit the models on the training data
xgb_reg_votes.fit(X_votes_train, y_votes_train)
xgb_reg_gross.fit(X_gross_train, y_gross_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=200, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [128]:
from sklearn.metrics import mean_squared_error

# Make predictions for votes and gross collection on the test data
y_votes_pred = xgb_reg_votes.predict(X_votes_test)
y_gross_pred = xgb_reg_gross.predict(X_gross_test)

# Calculate MSE for votes and gross collection
mse_votes = mean_squared_error(y_votes_test, y_votes_pred)
mse_gross = mean_squared_error(y_gross_test, y_gross_pred)

print("Mean Squared Error for Votes:", mse_votes)
print("Mean Squared Error for Gross Collection:", mse_gross)


Mean Squared Error for Votes: 110130750056.69978
Mean Squared Error for Gross Collection: 1.1900253680463974e+17


In [129]:
import joblib

# Save the best models for votes and gross collection
joblib.dump(xgb_reg_votes, 'best_model_votes.pkl')
joblib.dump(xgb_reg_gross, 'best_model_gross.pkl')

['best_model_gross.pkl']

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

# Load data from CSV file
data = pd.read_csv('merged_data_without_commas.csv')

# Split data into features (X) and labels (y) for votes and gross collection
X_votes = data[['Duration', 'Year', 'Ratings', 'Metascore']]
y_votes = data['Votes']

X_gross = data[['Duration', 'Year', 'Ratings', 'Metascore', 'Popularity']]
y_gross = data['Gross Collection']

# Handle missing values
X_votes.fillna(0, inplace=True)  # Replace NaN values with 0 for votes
X_gross.fillna(0, inplace=True)  # Replace NaN values with 0 for gross collection

# Scale the features
scaler_votes = StandardScaler()
X_votes_scaled = scaler_votes.fit_transform(X_votes)

scaler_gross = StandardScaler()
X_gross_scaled = scaler_gross.fit_transform(X_gross)

# Split data into training and testing sets
X_votes_train, X_votes_test, y_votes_train, y_votes_test = train_test_split(X_votes_scaled, y_votes, test_size=0.2, random_state=42)
X_gross_train, X_gross_test, y_gross_train, y_gross_test = train_test_split(X_gross_scaled, y_gross, test_size=0.2, random_state=42)

# Define the XGBoost regressor
xgb_reg_votes = XGBRegressor()
xgb_reg_gross = XGBRegressor()

# Define the hyperparameter grid for tuning
param_grid = {
    'learning_rate': [0.1, 0.01],
    'max_depth': [3, 5, 7],
    'n_estimators': [100, 200, 300]
}

# Perform hyperparameter tuning using GridSearchCV for votes
grid_search_votes = GridSearchCV(estimator=xgb_reg_votes, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search_votes.fit(X_votes_train, y_votes_train)

# Perform hyperparameter tuning using GridSearchCV for gross collection
grid_search_gross = GridSearchCV(estimator=xgb_reg_gross, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
grid_search_gross.fit(X_gross_train, y_gross_train)

# Get the best parameters
best_params_votes = grid_search_votes.best_params_
best_params_gross = grid_search_gross.best_params_

# Update the models with the best parameters
xgb_reg_votes = XGBRegressor(**best_params_votes)
xgb_reg_gross = XGBRegressor(**best_params_gross)

# Fit the models on the training data
xgb_reg_votes.fit(X_votes_train, y_votes_train)
xgb_reg_gross.fit(X_gross_train, y_gross_train)

# Make predictions for votes and gross collection on the test data
y_votes_pred = xgb_reg_votes.predict(X_votes_test)
y_gross_pred = xgb_reg_gross.predict(X_gross_test)

# Calculate MSE for votes and gross collection
mse_votes = mean_squared_error(y_votes_test, y_votes_pred)
mse_gross = mean_squared_error(y_gross_test, y_gross_pred)

# Print the Mean Squared Error for votes and gross collection
print("Votes Model:")
print("Mean Squared Error (MSE) for Votes:", mse_votes)

print("\nGross Collection Model:")
print("Mean Squared Error (MSE) for Gross Collection:", mse_gross)

# Save the best models for votes and gross collection
import joblib
joblib.dump(xgb_reg_votes, 'best_model_votes.pkl')
joblib.dump(xgb_reg_gross, 'best_model_gross.pkl')


C:\Users\write\AppData\Local\Temp\ipykernel_9592\78041720.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_votes.fillna(0, inplace=True)  # Replace NaN values with 0 for votes
C:\Users\write\AppData\Local\Temp\ipykernel_9592\78041720.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_gross.fillna(0, inplace=True)  # Replace NaN values with 0 for gross collection


Votes Model:
Mean Squared Error (MSE) for Votes: 110130750056.69978

Gross Collection Model:
Mean Squared Error (MSE) for Gross Collection: 1.1900253680463974e+17


['best_model_gross.pkl']

In [ ]:
Evaluation of the Model for Votes Prediction:

Mean Squared Error (MSE) for Votes: 110,130,750,056.70
Root Mean Squared Error (RMSE) for Votes: 331,865.61
R-squared (R2) for Votes: 0.80
Evaluation of the Model for Gross Collection Prediction:

Mean Squared Error (MSE) for Gross Collection: 119,002,536,804,639,740
Root Mean Squared Error (RMSE) for Gross Collection: 345,067,815.43
R-squared (R2) for Gross Collection: -0.038

Comparison of Model Performance:
The XGBoost model for votes prediction demonstrated a relatively good performance with an R-squared value of approx. 0.80.
This suggests that about 80% of the variance in votes can be explained by the model. 
The RMSE is also relatively low, indicating accurate predictive performance for votes.

On the other hand, the XGBoost model for gross collection prediction performed poorly,with an R-squared value of approx -0.038. 
This indicates that the model is not a good fit for the data, and its predictions for gross collection are not accurate. 
The high RMSE further supports this observation.

Based on the evaluation,it is evident that the XGBoost model for votes prediction outperforms the model for 
gross collection prediction. Therefore, the model for votes prediction (best_model_votes.pkl) is preferred due to its 
higher R-squared value and lower RMSE, which signify better predictive performance for votes.

It is important to further investigate and possibly improve the model for gross collection prediction by considering 
various factors such as data preprocessing, feature engineering, or exploring different modeling techniques. 
Until then, the current results suggest that the model for votes prediction is the better choice for making 
accurate predictions.